In [3]:
import numpy as np
import pandas as pd
import os
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping
import joblib

# === CONFIG ===
look_back = 24
data_dir = r"C:\Users\Hp\Downloads"
target_col = 0  # Index of the target column

# === Load Data ===
train = pd.read_csv(os.path.join(data_dir, "train_data.csv"))
val = pd.read_csv(os.path.join(data_dir, "val_data.csv"))
test = pd.read_csv(os.path.join(data_dir, "test_data.csv"))

# === Prepare Sequences ===
def create_sequences(data, look_back, target_col):
    X, y = [], []
    for i in range(len(data) - look_back):
        X.append(data.iloc[i:i+look_back].values)
        y.append(data.iloc[i+look_back, target_col])
    return np.array(X), np.array(y)

X_train, y_train = create_sequences(train, look_back, target_col)
X_val, y_val = create_sequences(val, look_back, target_col)
X_test, y_test = create_sequences(test, look_back, target_col)

# === Build LSTM Model ===
model = Sequential([
    LSTM(64, activation='tanh', input_shape=(look_back, X_train.shape[2])),
    Dense(32, activation='relu'),
    Dense(1)
])
model.compile(optimizer='adam', loss='mse')

# === Train ===
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=10,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)

# === Evaluate ===
val_loss = model.evaluate(X_val, y_val)
test_loss = model.evaluate(X_test, y_test)
print(f"Validation RMSE: {np.sqrt(val_loss):.4f}")
print(f"Test RMSE: {np.sqrt(test_loss):.4f}")

# === Save Model ===
model.save(os.path.join(data_dir, "lstm_model.h5"))
print("✅ LSTM model saved.")


D:\newmll\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 11s 18ms/step - loss: 0.0123 - val_loss: 0.0028
Epoch 2/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - loss: 8.2707e-04 - val_loss: 0.0020
Epoch 3/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - loss: 4.3078e-04 - val_loss: 0.0013
Epoch 4/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - loss: 3.0238e-04 - val_loss: 0.0013
Epoch 5/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - loss: 2.1310e-04 - val_loss: 0.0011
Epoch 6/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - loss: 1.6265e-04 - val_loss: 9.5292e-04
Epoch 7/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - loss: 1.3577e-04 - val_loss: 8.8888e-04
Epoch 8/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - loss: 1.0890e-04 - val_loss: 7.9023e-04
Epoch 9/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - loss: 9.5671e-05 - val_loss: 7.0149e-04
Epoch 10/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - loss: 8.2517e-05 - val_loss: 6.8384e-04
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 5.1350e-04 
27/27 ━━━━━━━

Validation RMSE: 0.0262
Test RMSE: 0.0219
✅ LSTM model saved.
